# Building Simple VFX Assets

This notebook is an adaptation of [Paolo Selva's VFX Intent](https://github.com/paoloemilioselva/assets/tree/structure-for-first-intent/intent-vfx-v1.0), meant to serve as a small example of how notesbooks can be used to demonstrate asset workflows. This notebook is designed to be run locally, ideally in an environment with `usdview`. 

## Step Zero: Setup

This notebook relies on two sets of utility functions:
1. Some USD-wrangling utils, which have been set up in `./src/utils.py`
2. Some utilities for downloading individual assets from [usd-wg/assets](https://github.com/usd-wg/assets), which have been set up in `./src/pull.py`
 
The code block below handles the setup for that, and includes some global flags for asset handling. 

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import subprocess

local_scripts_path = os.path.abspath(os.path.join('../src')) 
sys.path.insert(0, local_scripts_path)

LOCAL_ASSETS = False
OPEN_USDVIEW = False
PRINT_LAYER = False

OUT_DIR = "./out/"

if not LOCAL_ASSETS:
    from src import pull
    pull.teapot()


## Setup Geometry

In [ ]:
import subprocess
from pxr import Sdf, Usd, UsdGeom, UsdShade
from src import utils

# main details for current asset
#
asset_name = "teapot"
root_path = Sdf.Path.absoluteRootPath
root_asset_path = root_path.AppendChild(asset_name)

# build lower-level usd file with actual geometry
#
geo_filename = 'geo.usd'
default_shapes = []
with utils.make_stage(OUT_DIR + geo_filename) as stage:
    # build root
    root = utils.build_asset_root(stage, root_asset_path)
    # at geometry level we apply the GeomModelAPI schema
    UsdGeom.ModelAPI.Apply(root.GetPrim())
    # the geo prim to hold proxy and render prims
    geo_scope = UsdGeom.Scope.Define(stage, root.GetPath().AppendChild("geo"))
    # default purpose prim (no proxy or render)
    default = UsdGeom.Scope.Define(stage, geo_scope.GetPath().AppendChild("default"))
    default.GetPrim().GetAttribute("purpose").Set("default")

    asset_path = ""
    if LOCAL_ASSETS:
        asset_path = "../in/geo.usd"
    else:
        asset_path = "../../../assets/UtahTeapot.usd"
    default.GetPrim().GetReferences().AddReference(asset_path)

    # get default-shapes for later binding
    utils.find_type_or_kind(default_shapes, default.GetPrim(), "Mesh", "", False)
    print(default_shapes)
    # set default prim and close stage
    stage.SetDefaultPrim(root.GetPrim())
    
    if PRINT_LAYER:
        print(stage.ExportToString())

    if OPEN_USDVIEW:
        subprocess.run(['usdview', OUT_DIR + geo_filename], shell=True)
        


## Setup Materials

In [3]:
# build material layer
#
mtl_filename = 'mtl.usd'
with utils.make_stage(OUT_DIR + mtl_filename) as stage:
    # build root prim
    root = utils.build_asset_root(stage, root_asset_path)
    # add geometry reference to root prim
    root.GetPrim().GetReferences().AddReference( "./" + geo_filename)
    # create mtl scope to hold all materials for proxy and render purposes
    mtl_scope = UsdGeom.Scope.Define(stage, root.GetPath().AppendChild("mtl"))
    # create material and shader for purposes and bind to shapes
    default_material = utils.build_material(stage,mtl_scope,"default_material")
    default_shader_mtlx = utils.build_mtlx_standard_surface(stage,default_material,"default_shader_mtlx")
    default_shader = utils.build_preview_shader(stage,default_material,"default_shader")
    for p in default_shapes:
        p_over = stage.OverridePrim(p)
        UsdShade.MaterialBindingAPI.Apply(p_over)
        UsdShade.MaterialBindingAPI(p_over).Bind(default_material)
    # set default prima and close stage
    stage.SetDefaultPrim(root.GetPrim())

    if PRINT_LAYER:
        print(stage.ExportToString())

    if OPEN_USDVIEW:
        subprocess.run(['usdview', OUT_DIR + mtl_filename], shell=True)


## Setup Payload

In [ ]:
# build top-level payload with geometry and material
#
payload_filename = 'payload.usd'
with utils.make_stage(OUT_DIR + payload_filename) as stage:
    # add material layer to sublayers
    stage.GetRootLayer().subLayerPaths = [ "./" + mtl_filename]
    # create root prim
    root = utils.build_asset_root(stage, root_asset_path)
    # set default prim and close the stage
    stage.SetDefaultPrim(root.GetPrim())

    if PRINT_LAYER:
        print(stage.ExportToString())

    if OPEN_USDVIEW:
        subprocess.run(['usdview', OUT_DIR + payload_filename], shell=True)

## Assemble File

In [5]:
# build main asset file
#
asset_filename = '{}.usd'.format(asset_name)
with utils.make_stage(OUT_DIR + asset_filename) as stage:
    # build root prim with metadata for current asset
    root = utils.build_asset_root(stage, root_asset_path, 
                            i_asset_name=asset_name, 
                            i_asset_identifier= "./" + payload_filename, 
                            i_asset_version="1.0")
    # add payload to root layer to material and geometry are payloaded
    root.GetPrim().GetPayloads().AddPayload( "./" + payload_filename)
    # set default prim and close the stage
    stage.SetDefaultPrim(root.GetPrim())

    if PRINT_LAYER:
        print(stage.ExportToString())

    if OPEN_USDVIEW:
        subprocess.run(['usdview', OUT_DIR + asset_filename], shell=True)